In [3]:
from bs4 import BeautifulSoup
import configparser
import requests
import json
import pickle
import pandas as pd
from constants_funcs import get_session, int_or_float_or_str

In [4]:
LLHEADER = 'https://www.learnedleague.com'
LOGINFILE = LLHEADER + '/ucp.php?mode=login'
USER_DATA = LLHEADER + '/profiles/previous.php?%s'
QHIST = LLHEADER + '/profiles/qhist.php?%s'
MATCH_DATA = LLHEADER + '/match.php?%s'
ONEDAYS = LLHEADER + '/oneday'
STANDINGS = '/standings.php?'
LLSTANDINGS = LLHEADER + STANDINGS
ARUNDLE = LLSTANDINGS + '%d&A_%s'
INPUTDATA = 'logindata.ini'
TOTAL_MATCHES_PER_SEASON = 25

In [5]:
sess = get_session()
sess

using existing login session


In [16]:
# Personal profile & data can be found here.
# main_data = sess.get('https://learnedleague.com/profiles.php?23168&9')
# html_text = main_data.text
# soup = BeautifulSoup(html_text, 'html.parser')
# # print(soup)

# qh = soup.find('div', attrs={'class': 'qhistory'}).find_all('ul', attrs={'class': 'mktree'})
# qh

AttributeError: 'NoneType' object has no attribute 'find_all'

In [8]:
# class Question:
#     def __init__(self, text, category, season, day, qnum, correct) -> None:
#         self.text = text
#         self.category = category
#         self.season = season
#         self.day = day
#         self.qnum = qnum
#         self.correct = correct

In [109]:
# q_arr = []
# for q_cat in qh:
#     catarr = q_cat.find_all('span', attrs={'class': 'catname'})
#     print(catarr)
#     if len(catarr) == 1:
#         category = q_cat.find('span', attrs={'class': 'catname'}).text
#         rows = q_cat.find('table', attrs={'class': 'qh'}).find_all('tr')[1:]
#         for q in rows:
#             md, txt, correct = q.find_all('td')
#             season, day, qnum = md.find_all('a')
#             season = int(season.text[2:])
#             day = int(day.text[2:])
#             qnum = int(qnum.text[1])
#             txt = txt.text
#             correct = True if correct['class'][1] == 'g' else False
#             q_arr.append(Question(txt, category, season, day, qnum, correct))

[<span class="catname">AMER HIST</span>]
[<span class="catname">ART</span>]
[<span class="catname">BUS/ECON</span>]
[<span class="catname">CLASS MUSIC</span>]
[<span class="catname">CURR EVENTS</span>]
[<span class="catname">FILM</span>]
[<span class="catname">FOOD/DRINK</span>]
[<span class="catname">GAMES/SPORT</span>]
[<span class="catname">GEOGRAPHY</span>]
[<span class="catname">LANGUAGE</span>]
[<span class="catname">LIFESTYLE</span>]
[<span class="catname">LITERATURE</span>]
[<span class="catname">MATH</span>]
[<span class="catname">POP MUSIC</span>]
[<span class="catname">SCIENCE</span>]
[<span class="catname">TELEVISION</span>]
[<span class="catname">THEATRE</span>]
[<span class="catname">WORLD HIST</span>]


In [113]:
q_arr[98].__dict__

{'text': 'A group of women living in and around the Gee\'s Bend hamlet in the Black Belt of Alabama are renowned for their works of art using what specific medium—a medium that is also referenced in the title of a 1985 piece by American artist Faith Ringgold, "Street Story <Redacted>"?',
 'category': 'ART',
 'season': 88,
 'day': 6,
 'qnum': 2,
 'correct': False}

In [114]:
dicts = [q.__dict__ for q in q_arr]
with open('my_q_history.json', 'w') as f:
    json.dump(dicts, f)

In [ ]:
class Question:
    def __init__(self, text, category, season, day, qnum, correct) -> None:
        self.text = text
        self.category = category
        self.season = season
        self.day = day
        self.qnum = qnum
        self.correct = correct
full_question_history = []

In [19]:
# Get the questions and answers for each matchday in past seasons.
question_history = dict() # format: {season-md-qnum  (str): {
#  season: int, matchday: int, qnum: int, question: str, answer: str, category: str, forfeit: float, tot_correct: int
#   A_correct: int, B_correct: int, C_correct: int, D_correct: int, E_correct: int, R_correct: int, defense: float
# }}
# a dict of dicts.
def get_question(season: int, md: int) -> dict:
    if not isinstance(season, int) or not 52 <= season <= 93:
        print('valid seasons are integer 52-93')
        return
    elif not isinstance(md, int) or not 1 <= md <= 25:
        print('valid matchdays are integers 1-25')
        return
    # early template (seasons 52-56): https://learnedleague.com/ll55/questions/md01.php
    # late template (seasons 57-present): https://learnedleague.com/match.php?60&1
    if season <= 56:
        url = f"https://learnedleague.com/ll{season}/questions/md{str(md).zfill(2)}.php"
    else:
        url = f"https://learnedleague.com/match.php?{season}&{md}"
    main_data = sess.get(url)
    html_text = main_data.text
    soup = BeautifulSoup(html_text, 'html.parser')
    qs = soup.findAll('div', attrs={'class': 'ind-Q20'})
    # anss = soup.findAll('div', attrs={'class': 'answer3'})

    for idx, q in enumerate(qs[:6]):
        txt = q.text
        # qnum = int(txt[2])
        qnum = idx + 1
        txt = txt[4:].strip()
        cat = txt.split('-')[0].strip()
        txt = '-'.join(txt.split('-')[1:]).strip()
        extra = ''
        if season > 56: extra = '1'
        elif idx >= 6: extra = str(idx//6 + 1)
        ans = soup.find('div', attrs={'id': f"Q{idx%6+1}{extra}ANS"}).text.strip()
        # if season <= 55:
        #     ans = soup.find('div', attrs={'id': f"Q{idx+1}ANS"}).text.strip()
        # else:
        #     ans = soup.find('div', attrs={'id': f"Q{idx+1}1ANS"}).text.strip()
            # ans = anss[idx].text.split('\n\t')[1].strip()
        question_history[f"{season}-{md}-{qnum}"] = {
            'season': season, 'matchday': md, 'qnum': qnum, 'question': txt, 'answer': ans, 'category': cat
        }
    if season >= 60: # newer seasons
        metrics = soup.find('div', attrs={'id': 'rght'})
        for lvl in ['A', 'B', 'C', 'D', 'E', 'R']:
            lvl_data = [x.text for x in metrics.findAll('td', attrs={'class': f'level{lvl}'})]
            for idx in range(1,7):
                question_history[f"{season}-{md}-{idx}"][f"{lvl}_correct"] = int(lvl_data[idx+1])
        leaguewide, defense = metrics.find('tfoot').findAll('tr')
        leaguewide, defense = leaguewide.findAll('td'), defense.findAll('td')
        for idx in range(1,7):
            question_history[f"{season}-{md}-{idx}"]['tot_correct'] = int(leaguewide[idx+1].text)
            question_history[f"{season}-{md}-{idx}"]['forfeit'] = float(leaguewide[1].text)
            question_history[f"{season}-{md}-{idx}"]['defense'] = float(defense[idx+1].text)
    else:
        for row in soup.find('table', attrs={'class': 'ind-boxATbl2'}).findAll('tr'):
            row = [x.text.strip() for x in row.findAll('td')]
            if row[0].startswith('All ') and (len(row[0]) == 5 or row[0].endswith('(Class 1)')):
                for idx in range(1,7):
                    question_history[f"{season}-{md}-{idx}"][f"{row[0][4]}_correct"] = int(row[idx+1])
            elif row[0] == 'Totals' or row[0] == 'Leaguewide' or row[0] == 'Leaguewide: Class 1':
                for idx in range(1,7):
                    question_history[f"{season}-{md}-{idx}"]["tot_correct"] = int(row[idx+1])
                    question_history[f"{season}-{md}-{idx}"]["forfeit"] = float(row[1])
            elif row[0] == 'Defense' or row[0] == 'Defense: Class 1':
                for idx in range(1,7):
                    question_history[f"{season}-{md}-{idx}"]["defense"] = float(row[idx+1])
    return { x: question_history[f"{season}-{md}-{x}"] for x in range(1,7)}
# ans = soup.find('div', attrs={'id': 'xyz'}).text.strip()
# level_stats = [row.find_all('td') for row in 
#     soup.find('table', attrs={'class': 'MDTable sortable'}).findAll('tr')[1:]]
# d = {str(lvl.text): int(pct.text) for lvl, pct in level_stats}
# q


In [20]:
# for seas in range(52, 94): # getting questions from seasons 52-93
for day in range(1, 26): # matchdays 1-25
    get_question(93, day)
print(f"done with season {93}")

done with season 93


In [259]:
with open('questions.pkl', 'wb') as fp:
    pickle.dump(question_history, fp, protocol=pickle.HIGHEST_PROTOCOL)

In [21]:
with open('questions.pkl', 'rb') as fp:
    full_question_history = pickle.load(fp)

In [22]:
df = pd.DataFrame.from_dict(full_question_history, orient='index')
df.head()

,season,matchday,qnum,question,answer,category,A_correct,B_correct,C_correct,D_correct,E_correct,R_correct,tot_correct,forfeit,defense
59-1-1,59,1,1,The formal agreement known today as the Tripar...,"GERMANY, ITALY, JAPAN",WORLD HIST,88,82,71,59,35,50,60,4.0,0.9
59-1-2,59,1,2,The domicile pictured here features prominentl...,AMERICAN GOTHIC,ART,90,85,82,77,50,75,74,4.0,1.4
59-1-3,59,1,3,"While Fănică Luca, Dalila Cernătescu, and Simi...",GHEORGHE ZAMFIR,CLASS MUSIC,74,56,49,31,19,30,38,4.0,2.0
59-1-4,59,1,4,"Name the man, a pivotal figure in the history ...",CURT FLOOD,GAMES/SPORT,59,52,39,27,15,22,31,4.0,1.6
59-1-5,59,1,5,"Schlemiel! Schlimazel! This dish is a thick, h...",HASENPFEFFER,FOOD/DRINK,89,73,63,47,26,41,51,4.0,1.4


In [25]:
new_season = pd.DataFrame.from_dict(question_history, orient='index')
df = pd.concat([df, new_season])
df.head()

,season,matchday,qnum,question,answer,category,A_correct,B_correct,C_correct,D_correct,E_correct,R_correct,tot_correct,forfeit,defense
59-1-1,59,1,1,The formal agreement known today as the Tripar...,"GERMANY, ITALY, JAPAN",WORLD HIST,88,82,71,59,35,50,60,4.0,0.9
59-1-2,59,1,2,The domicile pictured here features prominentl...,AMERICAN GOTHIC,ART,90,85,82,77,50,75,74,4.0,1.4
59-1-3,59,1,3,"While Fănică Luca, Dalila Cernătescu, and Simi...",GHEORGHE ZAMFIR,CLASS MUSIC,74,56,49,31,19,30,38,4.0,2.0
59-1-4,59,1,4,"Name the man, a pivotal figure in the history ...",CURT FLOOD,GAMES/SPORT,59,52,39,27,15,22,31,4.0,1.6
59-1-5,59,1,5,"Schlemiel! Schlimazel! This dish is a thick, h...",HASENPFEFFER,FOOD/DRINK,89,73,63,47,26,41,51,4.0,1.4


In [26]:
df.to_csv('FULL_QUESTION_HISTORY.csv')